In [1]:
from google.colab import drive

drive.mount('/content/gdrive')
root_path = 'gdrive/My Drive//content/gdrive/My Drive/Colab Notebooks/' 


Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [2]:
from sklearn.preprocessing import OneHotEncoder, LabelEncoder, StandardScaler
from fancyimpute import IterativeImputer
import pandas as pd
import numpy as np
import pickle
#Using TensorFlow backend.
import tensorflow as tf
print(tf.__version__)

/usr/local/lib/python3.6/dist-packages/sklearn/externals/six.py:31: FutureWarning: The module is deprecated in version 0.21 and will be removed in version 0.23 since we've dropped support for Python 2.7. Please rely on the official version of six (https://pypi.org/project/six/).
  "(https://pypi.org/project/six/).", FutureWarning)
Using TensorFlow backend.


2.2.0-rc3


In [0]:
#Read csv
raw_df = pd.read_csv('/content/gdrive/My Drive/Colab Notebooks/exercise_05_test.csv')

In [0]:
#Seperate categorical variables
category_df = raw_df.loc[:,raw_df.dtypes == np.object]

In [11]:
#Seperate numerical variables
num_df = raw_df.loc[:,raw_df.dtypes == np.float64]
num_df.head()

,x0,x1,x2,x3,x4,x5,x6,x7,x8,x9,x10,x11,x12,x13,x14,x15,x16,x17,x18,x19,x20,x21,x22,x23,x24,x25,x26,x27,x28,x29,x30,x31,x32,x33,x36,x37,x38,x39,x40,x42,...,x58,x59,x60,x61,x62,x63,x64,x65,x66,x67,x69,x70,x71,x72,x73,x74,x75,x76,x77,x78,x79,x80,x81,x82,x83,x84,x85,x86,x87,x88,x89,x90,x91,x92,x94,x95,x96,x97,x98,x99
0,0.519093,-4.606038,13.707586,-17.990903,12.873394,14.910935,2.915341,-10.110081,1.628317,-0.365064,10.646442,3.922680,34.998362,2.433472,-3.858548,0.104192,-5.519994,3.467700,31.000121,-4.590834,0.086110,6.255197,4.697129,2.180207,7.108676,-0.908606,-0.515504,45.331378,1.637379,-9.039186,-0.790305,2.265819,-4.036708,6.636670,0.220489,-1.358649,2.930821,-6.678993,43.244917,-3.146405,...,45.348605,1.072083,-4.914587,-14.550192,-5.232047,-26.134083,14.509004,0.416692,-22.838256,-2.857738,-10.884239,3.085385,63.367373,4.372968,-14.474131,-46.024063,2.540470,10.338857,12.721655,1.992730,-19.241954,-47.967821,8.581891,0.996542,10.724987,7.760128,5.567248,2.267702,0.024121,-0.532450,-5.916457,-138.889799,-0.246395,-1.912581,-0.633811,1.181750,-0.836542,-2.144871,0.010353,-4.819828
1,-12.357004,13.874141,14.052924,34.129247,34.511107,34.583336,-0.482540,-6.583407,-4.326799,-1.216928,-5.709141,6.175520,2.121554,1.578756,2.038905,-5.592737,-2.158564,0.521756,37.805522,1.901376,37.967075,4.546150,8.383848,9.278303,-7.514222,-1.407846,-0.761477,-18.080597,0.510580,4.201491,-0.544123,1.078374,2.652454,1.915220,0.874948,2.237308,-2.800175,-2.606274,-2.164761,52.113208,...,21.351741,2.842510,-0.187795,1.657564,-12.863514,-14.917924,4.622873,-3.404009,-6.014435,6.476535,23.032601,0.134983,-39.146113,4.458458,-23.588318,-24.292462,-1.657806,-2.733824,15.606533,0.216613,25.813679,59.937780,0.012385,0.107180,-20.315577,-8.655952,-4.170537,8.350734,13.554121,3.673744,-1.785349,117.305737,0.316266,6.838756,-4.357530,0.390822,14.052604,-2.744911,-0.005648,-1.253519
2,1.834922,2.665252,-44.873210,21.941920,10.102981,5.962249,-5.733909,-4.061670,-0.172269,0.096051,22.315785,3.330807,-8.121101,-1.985411,6.428962,-2.646925,8.367280,-5.545219,5.879821,10.221974,-9.095619,-1.368356,-39.691225,5.286057,0.562340,0.016971,1.894738,10.513043,1.026482,-7.617738,9.442215,2.980519,3.070543,-1.370332,-1.454560,-3.010714,1.356234,9.807104,2.126665,-26.994659,...,-22.040152,-4.361618,13.568331,12.005017,0.017017,-4.266916,-3.012223,-5.088874,9.907362,-12.322727,-28.776837,-1.321741,11.003258,-7.499675,22.115380,-37.524947,0.888089,0.662938,1.793714,-3.878713,-2.324871,-38.409542,1.975282,0.540509,31.884014,-3.407244,2.122876,-10.592297,-6.830781,0.551768,-2.317764,-66.548569,-0.657101,3.128596,1.880922,0.810115,15.737267,-9.031679,0.089295,-4.285350
3,20.972483,11.548506,-40.924625,-35.296796,-35.253101,-14.601890,5.045075,10.841771,-1.872260,-0.002583,60.212310,-11.716837,-17.011739,-1.363757,2.843387,-0.390018,-0.504431,-0.001356,-45.845038,-2.662568,-0.540813,11.362248,-19.387491,-2.160935,3.217326,-0.456019,0.762841,-33.240271,-1.254007,-4.757926,-2.848993,3.598553,-1.466811,-6.091896,-0.387709,1.472792,2.963676,-11.401523,-10.641658,-68.793716,...,-71.731797,-7.243651,5.103674,-2.190475,-5.299044,-1.243398,15.140766,3.999990,36.387867,-5.651444,-17.961932,2.802529,9.772352,4.565211,49.274375,-22.745785,6.892522,4.513212,-13.017882,-0.429544,26.759971,-16.694840,0.824581,-0.854778,14.052699,8.700661,-2.650097,10.092507,-12.210058,0.939368,5.234044,-150.133466,-0.391609,-4.306940,-2.326108,-1.968523,-4.292831,-1.674606,-0.088565,-0.981937
4,-9.916044,5.509811,31.749288,-0.803916,-4.005098,20.912490,0.419346,-2.949516,1.057176,-0.338547,25.056651,3.103853,-7.541111,1.521165,12.184929,-2.534174,2.732948,3.906203,-6.551110,-2.489374,8.318352,2.012928,2.698287,8.943937,-2.595725,0.148272,0.672342,-24.317962,-0.897424,-6.902125,-11.206673,-1.611313,-0.503040,7.663023,0.340081,-3.284034,3.318112,-1.255465,21.673715,22.806935,...,-27.620373,2.486901,1.339638,4.196005,-5.997144,-24.016444,-3.917210,2.905353,-8.326432,5.396941,15.303420,3.057188,32.200252,12.

In [0]:
import os
os.chdir('/content/gdrive/My Drive/Colab Notebooks')
os.getcwd()
#Open models
with open('imp', 'rb') as i:
    imp = pickle.load(i)
    
with open('le_dict', 'rb') as l:
    le_dict = pickle.load(l)
    
with open('enc', 'rb') as e:
    enc = pickle.load(e)

with open('scalar', 'rb') as s:
    scalar = pickle.load(s)

In [15]:
#convert categorical vars ($,%) to numeric
category_df['x41'] = category_df['x41'].str.replace('$','').astype(float)
category_df['x45'] = category_df['x45'].str.replace('%','').astype(float)


KeyError: ignored

In [16]:
#add converted vars to numeric df
num_df['x41'] = category_df['x41']
num_df['x45'] = category_df['x45']
category_df.drop(['x41','x45'],axis = 1, inplace = True)

KeyError: ignored

In [0]:
#create mask to align same categories with different names
day_mask1 = category_df['x35'] == 'wednesday'
day_mask2 = category_df['x35'] == 'fri'
day_mask3 = category_df['x35']  == 'thur'


In [18]:
#align names
category_df['x35'][day_mask1] = 'wed'
category_df['x35'][day_mask2] = 'friday'
category_df['x35'][day_mask3] = 'thurday'

/usr/local/lib/python3.6/dist-packages/pandas/core/generic.py:8765: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._update_inplace(new_data)


In [19]:
#perform inductive, single imputation with numerical df
#NOTE: Will not attempt multiple imputation
imp_num_df = pd.DataFrame(imp.transform(num_df))
imp_num_df.index = num_df.index
imp_num_df.columns = num_df.columns

/usr/local/lib/python3.6/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function safe_indexing is deprecated; safe_indexing is deprecated in version 0.22 and will be removed in version 0.24.
  warnings.warn(msg, category=FutureWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function safe_indexing is deprecated; safe_indexing is deprecated in version 0.22 and will be removed in version 0.24.
  warnings.warn(msg, category=FutureWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function safe_indexing is deprecated; safe_indexing is deprecated in version 0.22 and will be removed in version 0.24.
  warnings.warn(msg, category=FutureWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function safe_indexing is deprecated; safe_indexing is deprecated in version 0.22 and will be removed in version 0.24.
  warnings.warn(msg, category=FutureWarni

In [20]:
#impute missing categorical vars as most frequent in category
for col in category_df:
    max_freq = category_df[col].value_counts().index[0]
    category_df[col][pd.isna(category_df[col])] = max_freq

/usr/local/lib/python3.6/dist-packages/pandas/core/generic.py:8765: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._update_inplace(new_data)


In [0]:
#convert categories into nominal integers
label_df = pd.DataFrame()
for col in category_df:
    le = le_dict[col]
    label_df[col] = le.transform(category_df[col])

In [0]:
#create df of binary cols representing instance of each category across multipile columns
onehot_df = pd.DataFrame(enc.transform(label_df).toarray())
#feature_names = enc.get_feature_names()
#onehot_df.columns = feature_names

In [0]:
scaled_num_df = pd.DataFrame(scalar.transform(imp_num_df))

In [0]:
x_df = pd.concat([scaled_num_df, onehot_df], axis = 1)

In [0]:
#save x vars
with open('x_df_test', 'wb') as x:
    pickle.dump(x_df, x, pickle.HIGHEST_PROTOCOL)